In [2]:
import numpy as np
import pandas as pd
import json

In [4]:

with open('intents.json') as file:
  data = json.load(file)


In [5]:
data['intents']
tag = []
patterns = []
responses = []
for example in data['intents']:
  responses.append(example['responses'])
  for x in example['patterns']:
    patterns.append(x)
    tag.append(example['tag'])

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
tokenizer = Tokenizer(num_words=1000, oov_token="<UNK>")
tokenizer.fit_on_texts(patterns)


In [9]:
sequences = tokenizer.texts_to_sequences(patterns)
sequences

[[129],
 [11, 12, 6],
 [2, 130, 37],
 [131],
 [58, 95],
 [132, 96],
 [11, 12, 133],
 [134],
 [135],
 [],
 [136],
 [97, 6],
 [73, 73],
 [97, 6, 98],
 [137],
 [16, 74, 138],
 [73],
 [38, 75, 58, 95],
 [139, 19, 6, 98],
 [140],
 [16, 141, 19, 142],
 [143],
 [4, 2, 5, 17, 7, 15, 49],
 [4, 2, 5, 17, 7, 15, 59],
 [4, 2, 5, 17, 7, 5, 49],
 [4, 2, 5, 17, 7, 5, 59],
 [23, 99, 6],
 [15, 49],
 [15, 59],
 [23, 12, 15, 49],
 [49],
 [6, 12, 76, 100],
 [6, 12, 76, 100, 144],
 [23, 99, 6],
 [23, 145, 6],
 [59],
 [23, 76, 6],
 [23, 146, 6],
 [17],
 [15, 17],
 [24, 6, 38, 75, 17],
 [4, 12, 6, 147],
 [4, 2, 15, 17],
 [4, 60, 16, 61, 6],
 [77, 15, 17],
 [4, 12, 6],
 [23, 12, 6],
 [23, 2, 148],
 [4, 74, 16, 149, 19],
 [23, 74, 16, 150, 19],
 [4, 12, 6],
 [39, 7, 3],
 [4, 2, 3, 39],
 [151, 152],
 [20, 12, 6, 101, 78],
 [4, 12, 15, 79],
 [79, 7, 153],
 [20, 2, 5, 3, 78],
 [3, 39],
 [4, 25, 3, 39],
 [2, 3, 78, 80, 154],
 [50, 102, 25, 3, 39],
 [4, 2, 5, 3, 79],
 [20, 60, 16, 155, 19, 3],
 [20, 60, 16, 156, 3]

In [10]:
final_sequences = pad_sequences(sequences,maxlen=20,truncating='post')
final_sequences

array([[  0,   0,   0, ...,   0,   0, 129],
       [  0,   0,   0, ...,  11,  12,   6],
       [  0,   0,   0, ...,   2, 130,  37],
       ...,
       [  0,   0,   0, ...,  24,  10,  31],
       [  0,   0,   0, ...,  18, 126,  31],
       [  0,   0,   0, ...,  60, 231,   6]])

In [11]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [12]:
le = LabelEncoder()
tags = le.fit_transform(tag)
labels = to_categorical(tags)


In [25]:
final_sequences.shape

(313, 20)

In [27]:
labels.shape

(313, 26)

In [14]:
import tensorflow as tf

# Define LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(26,)),  # Input
    tf.keras.layers.Embedding(1000,32),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(26,activation='softmax')
])

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 26, 32)         │        32,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 26)             │         1,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,522 (228.60 KB)

 Trainable params: 58,522 (228.60 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.fit(final_sequences,labels,epochs=200)

Epoch 1/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.0409 - loss: 3.2541 
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0762 - loss: 3.2141
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0985 - loss: 3.1727
Epoch 4/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1400 - loss: 3.1175
Epoch 5/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1107 - loss: 3.0876
Epoch 6/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1116 - loss: 3.0827
Epoch 7/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1683 - loss: 3.0389
Epoch 8/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2525 - loss: 2.9654
Epoch 9/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2058 - loss: 2.9132
Epoch 10/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2687 - loss: 2.8047
Epoch 11/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.2486 - loss: 2.6996
Epoch 12/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/ste

In [41]:
test = 'welcome'
test = tokenizer.texts_to_sequences([test])
test2 = pad_sequences(test,maxlen=20)
test2

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]],
      dtype=int32)

In [42]:
result = model.predict(test2)
output_class = np.argmax(result)
le.inverse_transform(np.array([output_class]))

1/1 [==============================] - 0s 27ms/step


array(['greeting'], dtype='<U14')

In [17]:
model.save('chatbot.h5')

In [44]:
import pickle

In [46]:

pickle.dump(tokenizer,open('tokenizer.pkl','wb'))

In [47]:
pickle.dump(le,open('labelencoder.pkl','wb'))

In [48]:
def chat():
  while True:
    text = input("user: ")
    text = tokenizer.texts_to_sequences([text])
    text = pad_sequences(text,maxlen=20)
    result = model.predict(text)
    output_class = np.argmax(result)
    output = le.inverse_transform(np.array([output_class]))
    for example in data['intents']:
      if example['tag'] == output[0]:
        print("Chatbot : ",np.random.choice(example['responses']))

chat()

user: hi 
1/1 [==============================] - 0s 24ms/step
Chatbot :  Hello!
user: i am sad
1/1 [==============================] - 0s 25ms/step
Chatbot :  Sad to see you go :(
user: what can you do for me 
1/1 [==============================] - 0s 28ms/step
Chatbot :  You can ask me questions regarding college, and i will try to answer them
user: what are departments exist 
1/1 [==============================] - 0s 24ms/step
Chatbot :  I'm Mind Reader
user: where us the college 
1/1 [==============================] - 0s 24ms/step
Chatbot :  https://www.google.com/maps/place/Cairo+University/@30.0272027,31.2085262,15z/data=!4m2!3m1!1s0x0:0xa8c3715257b6f3cb?sa=X&ved=1t:2428&ictx=111


KeyboardInterrupt: Interrupted by user